# GAN IMPLEMENTATION FROM COMMUNITY INQ28 MODELS
- Code Author: Jack MacCormick
- Instagram: argelpaints: https://www.instagram.com/argelpaints/
- Done in partnership with 28MAG: https://28-mag.com/

**GAN Training and Development Code:**

This project is the use of GAN style Machine Learning algorithms to generate new images, based off the works of the Warhammer INQ28 community. These images are the dreamings of an abhorent AI system, who's task is to understand "what is INQ28" it knows not the world, it's surroundings, only those offered to it, and the endless void

In [5]:
# import necessary packages
import os
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image
from torch.autograd import Variable
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pylab
import h5py

from PIL import Image
import pandas as pd
import tensorflow as tf
from tensorflow import keras

import numpy as np
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import random
from numpy.random import randint

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Concatenate

from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Model

from tqdm import tqdm 
from IPython import display 

In [6]:
# define the standalone discriminator model
def define_discriminator(in_shape=(200,200,1)):
	# weight initialization
	init = 'uniform'
	# define model
	model = Sequential()
	# downsample to 100x100
	model.add(Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init, input_shape=in_shape))
	model.add(LeakyReLU(alpha=0.2))
	# downsample to 50x50
	model.add(Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init))
	model.add(LeakyReLU(alpha=0.2))
	# classifier
	model.add(Flatten())
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model

In [7]:
def define_generator(latent_dim):
	# weight initialization
	init = 'uniform'
	# define model
	model = Sequential()
	# foundation for 50x50 image
	n_nodes = 256 * 50 * 50
	model.add(Dense(n_nodes, kernel_initializer=init, input_dim=latent_dim))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Reshape((50, 50, 256)))
	# upsample to 100x100
	model.add(Conv2DTranspose(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init))
	model.add(LeakyReLU(alpha=0.2))
	# upsample to 200x200
	model.add(Conv2DTranspose(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init))
	model.add(LeakyReLU(alpha=0.2))
	# output 28x28x1
	model.add(Conv2D(1, (7,7), activation='tanh', padding='same', kernel_initializer=init))
	return model

In [8]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(generator, discriminator):
	# make weights in the discriminator not trainable
	discriminator.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(generator)
	# add the discriminator
	model.add(discriminator)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

In [9]:
# default values of standardised images
preferred_width = 200
preferred_height = 200

In [10]:
# paths to take images from and to save to (after transformation to BnW)
pathSource = "C:\\Users\\DataSci\\Desktop\\DS_Testing\\warGAN\\igTest"
pathDest = "C:\\Users\\DataSci\\Desktop\\DS_Testing\\warGAN\\BnWImages"

# list all objects in pathSource
dir_list = os.listdir(pathSource)


In [11]:
"""
read in images from local machine
create list to contain all standardised images;
    values [0:1] for ML, instead of [0:255] for colour channels
for image in pathSource:
    convert them to BnW
    save that BnW image to pathDest
    then add standardised version to images[]
"""

images = []

# for image in pathSource, convert them to BnW, save that to pathDest, then add standardised version to images[]
for image in dir_list:
    img = Image.open(pathSource+"\\"+image).convert('L').resize((preferred_width, preferred_height), Image.ANTIALIAS)
    # img.show()
    imgArray = np.array(img) # int array of pixels
    imgArray = imgArray.astype(float)/255 # standardise values to be 0:255
    images.append(imgArray)
    img.save(pathDest+"\\BnW"+image, "JPEG")


"""
# diagnostic checks of images, checking shape and values look correct

# code
print(np.shape(images))
print(images[0])
print(np.shape(images[0]))
# end code
"""

(123, 200, 200)
[[0.78431373 0.78039216 0.78431373 ... 0.80784314 0.80784314 0.81176471]
 [0.78039216 0.78039216 0.78431373 ... 0.80784314 0.80784314 0.80784314]
 [0.78039216 0.78039216 0.78039216 ... 0.81176471 0.81176471 0.80784314]
 ...
 [0.94117647 0.94117647 0.9372549  ... 0.94901961 0.95294118 0.95294118]
 [0.94117647 0.94117647 0.9372549  ... 0.95686275 0.95294118 0.94901961]
 [0.94901961 0.94509804 0.94117647 ... 0.95686275 0.95294118 0.94901961]]
(200, 200)


In [12]:
# setting a param for proportion of data to keep for testing
testingSize = int(len(dir_list)/5)

X_testing = np.array(images[0:testingSize])
X_training = np.array(images[testingSize:len(images)])

"""
# this is all diagnostic checks to ensuredata is passed correctly to training and testing splits

# code:
print(np.shape(X_testing)) 
print(X_testing[1]) # checking it's correctly put something into the testing variable
print(np.shape(X_training))
# end code
"""

(24, 200, 200)
[[0.35686275 0.34901961 0.34509804 ... 0.32941176 0.32941176 0.32156863]
 [0.35686275 0.35294118 0.35294118 ... 0.3372549  0.33333333 0.32156863]
 [0.35686275 0.35294118 0.35686275 ... 0.33333333 0.3372549  0.33333333]
 ...
 [0.89803922 0.89803922 0.89803922 ... 0.89019608 0.89019608 0.89019608]
 [0.89803922 0.89803922 0.89803922 ... 0.88627451 0.88627451 0.89019608]
 [0.89803922 0.89803922 0.90196078 ... 0.88627451 0.88627451 0.88627451]]
(99, 200, 200)


In [14]:
# select real samples
def generate_real_samples(dataset, n_samples):
	# choose random instances
	ix = randint(0, dataset.shape[0], n_samples)
	# select images
	X = dataset[ix]
	# generate class labels
	y = ones((n_samples, 1))
	return X, y

In [15]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = np.random.uniform(low=0.0, high=1.0, size=(latent_dim, n_samples))
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

In [31]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
	# generate points in latent space
	x_input = abs(generate_latent_points(latent_dim, n_samples))
	# predict outputs
	X = generator.predict(x_input)
	# create class labels
	y = zeros((n_samples, 1))
	return X, y

In [32]:
"""
trains the generator and discriminator parts of the GAN
saves the net model to a .h5 file
"""

def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs, n_batch):
	# calculate the number of batches per epoch
	bat_per_epo = int(dataset.shape[0] / n_batch)
	# calculate the total iterations based on batch and epoch
	n_steps = bat_per_epo * n_epochs
	# calculate the number of samples in half a batch
	half_batch = int(n_batch / 2)
	# prepare lists for storing stats each iteration
	d1_hist, d2_hist, g_hist, a1_hist, a2_hist = list(), list(), list(), list(), list()
	# declear how long it's gonna run
	print(n_steps)
	# manually enumerate epochs
	for i in range(n_steps):
		# get randomly selected 'real' samples
		X_real, y_real = generate_real_samples(dataset, half_batch)
		# update discriminator model weights
		d_loss1, d_acc1 = d_model.train_on_batch(X_real, y_real)
		# generate 'fake' examples
		X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
		# update discriminator model weights
		d_loss2, d_acc2 = d_model.train_on_batch(X_fake, y_fake)
		# prepare points in latent space as input for the generator
		X_gan = generate_latent_points(latent_dim, n_batch)
		# create inverted labels for the fake samples
		y_gan = ones((n_batch, 1))
		# update the generator via the discriminator's error
		g_loss = gan_model.train_on_batch(X_gan, y_gan)
		# summarize loss on this batch
		print('>%d/%d, d1=%.3f, d2=%.3f g=%.3f, a1=%d, a2=%d' %
			(i+1, n_steps, d_loss1, d_loss2, g_loss, int(100*d_acc1), int(100*d_acc2)))
		# record history
		d1_hist.append(d_loss1)
		d2_hist.append(d_loss2)
		g_hist.append(g_loss)
		a1_hist.append(d_acc1)
		a2_hist.append(d_acc2)
		if (i%10==0): # save frequency, this should really be a smart value
			g_model.save('WARgenerator.h5')		
	print("finished training")
	# this is the generator that can be called externally
	g_model.save('WARgenerator.h5') 
	"""
	Save is also embed this within the loop every so many itterations
	so something is saved/retained if user needs to abort while long training processes are occuring or if something fails, if the kernel or local machine crashes or if local machine looses power
	"""

In [33]:
"""
set up the data into something the GAN can use in training
"""

X_train=np.reshape(X_training,(X_training.shape[0], X_training.shape[1],X_training.shape[2],1))

In [19]:
"""
set up for training and training call
this can take some time depending on number of epochs
recommend running on a GPU
"""

# training params
latent_dim = 50 # size of the latent space
n_epochs = 80 # number of "generations"
n_batch = 10 # batch (of images) size
# create the discriminator
discriminator = define_discriminator()
# create the generator
generator = define_generator(latent_dim)
# create the gan
gan_model = define_gan(generator, discriminator)
# load image data
dataset = X_train

# train model
train(generator, discriminator, gan_model, dataset, latent_dim, n_epochs, n_batch)

d1=0.691, d2=0.741 g=0.695, a1=100, a2=0
>59, d1=0.690, d2=0.731 g=0.699, a1=100, a2=0
>60, d1=0.680, d2=0.731 g=0.700, a1=100, a2=0
>61, d1=0.706, d2=0.734 g=0.691, a1=0, a2=0
>62, d1=0.678, d2=0.730 g=0.696, a1=100, a2=0
>63, d1=0.691, d2=0.724 g=0.712, a1=100, a2=0
>64, d1=0.718, d2=0.705 g=0.731, a1=0, a2=0
>65, d1=0.723, d2=0.702 g=0.717, a1=0, a2=0
>66, d1=0.710, d2=0.718 g=0.704, a1=0, a2=0
>67, d1=0.711, d2=0.716 g=0.720, a1=0, a2=0
>68, d1=0.730, d2=0.706 g=0.726, a1=0, a2=0
>69, d1=0.728, d2=0.706 g=0.723, a1=0, a2=0
>70, d1=0.701, d2=0.703 g=0.729, a1=0, a2=0
>71, d1=0.740, d2=0.728 g=0.711, a1=0, a2=0
>72, d1=0.718, d2=0.731 g=0.699, a1=0, a2=0
>73, d1=0.699, d2=0.731 g=0.703, a1=0, a2=0
>74, d1=0.702, d2=0.736 g=0.691, a1=0, a2=0
>75, d1=0.677, d2=0.740 g=0.686, a1=100, a2=0
>76, d1=0.679, d2=0.742 g=0.691, a1=100, a2=0
>77, d1=0.684, d2=0.742 g=0.688, a1=100, a2=0
>78, d1=0.681, d2=0.742 g=0.685, a1=100, a2=0
>79, d1=0.694, d2=0.733 g=0.702, a1=0, a2=0
>80, d1=0.693, d2=0

In [28]:
"""
getting the model to draw something,
this can be done seperately, if put in another file
"""

model = load_model('WARgenerator.h5', compile = False)
# generate images
latent_points = generate_latent_points(latent_dim, 1)
 # has quite the think about it
predictionInstance = model.predict(latent_points)
print("done thinking")

done thinking


In [29]:
"""
getting the prediction into something useful
"""

# convert floats from ranges 0:1 to 0:255, so they can be blackness values in an image
xout = predictionInstance*255 
# print(xout) # this is diagnostic
# np.shape(xout) # this is diagnostic
flatXOut = xout.flatten()  
# print(len(flatXOut)) # should be 40,000 # this is diagnostic
arr_2d = np.reshape(flatXOut, (200, 200))
# convert float array to ints
array_int = abs(np.array(arr_2d, dtype='int')) 
# print(np.shape(array_int)) # this is diagnostic
# array_int # this is diagnostic

In [30]:
"""
draw the image of the output array
"""

img = Image.fromarray(array_int)
img.show()

In [20]:
"""
diagnostic check of the output image
"""

print("Format: {0}\nSize: {1}\nMode: {2}".format(img.format, 
    img.size, img.mode)) 

Format: None
Size: (200, 200)
Mode: L


In [21]:
"""
convert the image file to a recognisable and portable format
write out the image file to somewhere in the local machine
"""

imgRGB = img.convert(mode="RGB")
imgRGB.save("C:\\Users\\DataSci\\Desktop\\DS_Testing\\warGAN\\output\\dream3.jpg")